In [2]:
import pandas as pd
import numpy as np
import glob
import os

from simpledbf import Dbf5

# Setting the input & output directory and also the file type
data_directory = r"E:\Script\GBM_Basin_RCP_Projection\Output\Basin_Wise_Projection\Merged"
output_directory = r"E:\Script\GBM_Basin_RCP_Projection\Output\Basin_Wise_Projection"

patterns = ["*_combined_with_grid_points.dbf", "*_combined_without_grid_points.dbf"]

# Iterating through the input directory and the getting projection file name
# and storing it in a list
source_file_list = []

for pattern in patterns:
    for file in glob.glob(os.path.join(data_directory, pattern)):
        file_path_list = file.split(os.sep)
        source_file_name = file_path_list[-1].split(pattern[1:])[0]
        
        if source_file_name in source_file_list:
            pass
        
        else:
            source_file_list.append(source_file_name)

            
# Iterating thorugh the projection name list and calculating the basin wise average value
for item in source_file_list:
    
    # Mean calculation for the basin that contains grid point
    with_grid_dbf = Dbf5(os.path.join(data_directory, '{}{}'.format(item, patterns[0][1:])))
    with_grid_df = with_grid_dbf.to_dataframe()
    with_grid_df.rename(columns={'Remarks' : 'Catchment'}, inplace = True)
    
    with_grid_mean_df = with_grid_df.iloc[: , 3:].groupby(["Catchment"]).mean().reset_index()
    
    
    # Mean calculation for the basin that does not contain grid points
    without_grid_dbf = Dbf5(os.path.join(data_directory, '{}{}'.format(item, patterns[1][1:])))
    without_grid_df = without_grid_dbf.to_dataframe()
    without_grid_df.rename(columns={'Remarks' : 'Catchment'}, inplace = True)
    
    without_grid_weighted_df = without_grid_df.iloc[:, 5:17].mul(without_grid_df['Weightage'], axis=0)
    without_grid_weighted_df['Catchment'] = without_grid_df['Catchment']
    without_grid_mean_df = without_grid_weighted_df.groupby(["Catchment"]).sum().reset_index()
    
    
    # Combining the dataframe to get all the basin projections summary and sorting them serially
    combined_mean_df = pd.concat([with_grid_mean_df, without_grid_mean_df])
    combined_mean_df.insert(loc = 0, column = 'Sl', value = combined_mean_df.iloc[:, 0].map(lambda x: int(str(x)[1:])))
    combined_mean_df.sort_values(by='Sl', ascending=True, inplace = True)
    
    # Saving the basin projection dataframe
    combined_mean_df.to_csv(os.path.join(output_directory, '{}_Basin_Wise_Summary.csv'.format(item)), 
                            float_format ='%.2f', index = False)
    
    print(item)

Base_Temperature_2
Base_Temperature
